In [1]:
# google drive のマウント
current_folder = "/content/drive/MyDrive/Colab Notebooks/yolox/YOLOX"
from google.colab import drive
drive.mount('/content/drive')
%cd $current_folder

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/yolox/YOLOX


In [5]:
# yolox.expで使用する loguru のインストール
!pip install loguru
# pytorch onnx moduleのインストール
!pip install onnx-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 918.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.2 MB/s eta 0:00:00


In [10]:
import numpy as np
import torch
import torch.onnx
from yolox.exp import get_exp

# 各種のバージョンを確認する。
!python --version
print(f"numpy {np.__version__}")
print(f"pytorch {torch.__version__}")

Python 3.10.6
numpy 1.22.4
pytorch 2.0.1+cu118


In [11]:
# translate
# 参考url: https://learn.microsoft.com/ja-jp/windows/ai/windows-ml/tutorials/pytorch-convert-model

input_size = [416,416]
model_name = "yolox-nano"
exp_filename = None
checkpoint_file = "../checkpoints/yolox_nano.pth"
output_filename = f"../yolox_nano_{'_'.join(map(str,input_size))}.onnx"

model = get_exp(exp_filename,model_name).get_model()

ckpt = torch.load(checkpoint_file,map_location="cpu")
model.load_state_dict(ckpt["model"])

model.eval()
dummy_input = torch.randn(1, 3,*input_size, requires_grad=True)
torch.onnx.export(model, # model being run
      dummy_input,       # model input (or a tuple for multiple inputs)
      output_filename,   # where to save the model
      export_params=True, # store the trained parameter weights inside the model file
      opset_version=10,   # the ONNX version to export the model to
      do_constant_folding=True,  # whether to execute constant folding for optimization
      input_names = ['modelInput'],   # the model's input names
      output_names = ['modelOutput'], # the model's output names
      dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes
                            'modelOutput' : {0 : 'batch_size'}})


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

